In [ ]:
%run VQAs_packages_rev.ipynb

In [ ]:
Case = 'HexTruss_consistent'  # 'HexTruss_consistent' or 'PlaneStrain_lumped'

In [ ]:
if Case == 'HexTruss_consistent':
    %run FEM_consistent_system_for_HexTruss.ipynb
    H = np.array(np.linalg.inv(MF_consistent_user)*KF)
if Case == 'PlaneStrain_lumped':
    %run FEM_lumped_system_for_PlaneStrain.ipynb
    H = np.array(np.linalg.inv(M)*K)
Test_A = H/np.linalg.norm(H)
print('condition number of normalized Hamiltonian: ', la.cond(Test_A))
print('L2 norm of normalized Hamiltonian: ', np.linalg.norm(Test_A))

In [ ]:
# Classical Solution

qubitOp = MatrixOp(primitive=Test_A)
qubits = qubitOp.num_qubits

npme = NumPyMinimumEigensolver()

result_npme = npme.compute_minimum_eigenvalue(operator=qubitOp)
ref_value = result_npme.eigenvalue.real
print(f'Reference value: {ref_value:.5f}')

In [ ]:
# VQE approach with Estimator on QPU:

from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session 

%config InlineBackend.figure_format='retina'

# Instantiate the Runtime service that gives access to the simulator 
service = QiskitRuntimeService(channel='ibm_quantum', 
    token='XXX')  # replace with user's token
backend = service.get_backend('ibmq_jakarta') # replace with available backends with service.backends()

# Specify the problem
observable = SparsePauliOp.from_operator(Test_A)
hamiltonian = observable

# Choice of ansatz
ansatz = TwoLocal(observable.num_qubits, rotation_blocks=["rz", "ry"], entanglement_blocks="cz", entanglement="linear", reps=1)
display(ansatz.decompose().draw("mpl", scale=0.6))

# Ansatz circuit is defined by a vector of parameters
num_params = ansatz.num_parameters
print(num_params)

# VQE cost function and minimization
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    energy = (
        estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    )
    return energy

# To begin the routine, we start by specifying a random initial set of parameters
x0 = 2 * np.pi * np.random.random(num_params)

# To save intermediate results from the iteration process,
# view the value of the cost function per iteration, 
# and to monitor the progress of the routine
def build_callback(ansatz, hamiltonian, estimator, callback_dict):
    """Return callback function that uses Estimator instance,
    and stores intermediate values into a dictionary.

    Parameters:
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance
        callback_dict (dict): Mutable dict for storing values

    Returns:
        Callable: Callback function object
    """

    def callback(current_vector):
        """Callback function storing previous solution vector,
        computing the intermediate cost value, and displaying number
        of completed iterations and average time per iteration.

        Values are stored in pre-defined 'callback_dict' dictionary.

        Parameters:
            current_vector (ndarray): Current vector of parameters
                                      returned by optimizer
        """
        # Keep track of the number of iterations
        callback_dict["iters"] += 1
        # Set the prev_vector to the latest one
        callback_dict["prev_vector"] = current_vector
        callback_dict["cost_history"].append(
            cost_func(current_vector, ansatz, hamiltonian, estimator)
        )
        # Grab the current time
        current_time = time.perf_counter()
        # Find the total time of the execute (after the 1st iteration)
        if callback_dict["iters"] > 1:
            callback_dict["_total_time"] += current_time - callback_dict["_prev_time"]
        # Set the previous time to the current time
        callback_dict["_prev_time"] = current_time
        # Compute the average time per iteration and round it
        time_str = (
            round(callback_dict["_total_time"] / (callback_dict["iters"] - 1), 2)
            if callback_dict["_total_time"]
            else "-"
        )
        # Print to screen on single line
        print(
            "Iters. done: {} [Avg. time per iter: {}]".format(
                callback_dict["iters"], time_str
            ),
            end="\r",
            flush=True,
        )
    return callback


# Conduct the experiment setting the callback argument in minimize with our function
callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
    "_total_time": 0,
    "_prev_time": None,
}

# Because we are sending a large number of jobs that we would like to execute together, 
# here we use a Session to execute all the generated circuits in one block. 
with Session(backend=backend, max_time=28800):
    estimator = Estimator(options={
        "shots": int(1e4), 
        "optimization_level": int(2), 
        # "resilience_level": int(2), # Uncomment if using noise mitigation
        # "noise_factors": (1, 3, 5), # Uncomment if using noise mitigation
        # "noise_amplifier": 'LocalFoldingAmplifier', "extrapolator": 'QuadraticExtrapolator', # Uncomment if using noise mitigation
        "max_execution_time": int(28800)})
    callback = build_callback(ansatz, hamiltonian, estimator, callback_dict)
    res = minimize(
        cost_func,
        x0,
        args=(ansatz, hamiltonian, estimator),
        method="cobyla",
        callback=callback,
    ) # args is the standard SciPy way to supply the additional parameters needed by the cost function
    

# If the procedure terminates correctly, then the prev_vector and iters values in our callback_dict dictionary
# should be equal to the solution vector and total number of function evaluations, respectively.
all(callback_dict["prev_vector"] == res.x)
callback_dict["iters"] == res.nfev

# View the progress towards convergence as monitored by the cost history at each iteration
fig, ax = plt.subplots()
ax.plot(range(callback_dict["iters"]), callback_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")

In [ ]:
# Apply ZNE noise mitigation for experimental data

reference_circuit = QuantumCircuit(observable.num_qubits)

variational_form = TwoLocal(observable.num_qubits, rotation_blocks=["rz", "ry"], entanglement_blocks="cz", 
                            entanglement="linear", reps=1)
ansatz = reference_circuit.compose(variational_form)
display(ansatz.decompose().draw('mpl', style={'name': 'textbook'}, plot_barriers=False, scale=0.6))

solution = ansatz.bind_parameters(res.x)
display(solution.decompose().draw('mpl', style={'name': 'textbook'}, plot_barriers=False, scale=0.6))

from qiskit.transpiler import CouplingMap
backend = service.backend("simulator_statevector")
coupling_list = [ [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], 
                 [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]]
coupling_map = CouplingMap(couplinglist=coupling_list)
display(coupling_map.draw())
# When the backend is a simulator and resilience_level == 3, a coupling map is required.

estimator = Estimator(session=backend,
                      options={
        "shots": int(1e4), 
        # "optimization_level": int(3), 
        "resilience_level": int(2), 
        # "noise_factors": (1, 3, 5), 
        # "noise_amplifier": 'LocalFoldingAmplifier', "extrapolator": 'QuadraticExtrapolator',
        "coupling_map": coupling_list,        
        "max_execution_time": int(28800)}
)
job = estimator.run(solution, observable, shots=10000)
result_ZNE = job.result()
values_ZNE = result_ZNE.values

print("Experiment complete.".ljust(30))

experimental_min_eigenvalue = values_ZNE[0]
print(f">>> Expectation value: {experimental_min_eigenvalue}")
